In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import shuffle
from scipy.integrate import ode, odeint
from scipy.sparse import csgraph
from scipy.signal import find_peaks
from scipy.spatial import distance
from scipy.stats import norm
from scipy.interpolate import interp1d
from matplotlib.pylab import rcParams
from matplotlib import animation
import pickle
import collections
from IPython.display import HTML

In [2]:
%matplotlib inline
rcParams['figure.figsize'] = 15, 6
rcParams['animation.html'] = 'html5'

In [3]:
def disarrange(a, axis=-1):
    """
    Shuffle `a` in-place along the given axis.

    Apply numpy.random.shuffle to the given axis of `a`.
    Each one-dimensional slice is shuffled independently.
    """
    b = a.swapaxes(axis, -1)
    # Shuffle `b` in-place along the last axis.  `b` is a view of `a`,
    # so `a` is shuffled in place, too.
    shp = b.shape[:-1]
    for ndx in np.ndindex(shp):
        np.random.shuffle(b[ndx])
    return

In [4]:
def RandAdj(N=500,K=500):
    #N*N*K -1 1 Tensor
    u=np.array([[[-1]*int(K/2)+[1]*int(K/2)]*N]*N)
    disarrange(u)
    uu= (np.swapaxes(u,0,1)*u)*[[(((-1)**(np.arange(K)+1)))]]
    JN= np.sum(uu[:,:,:100],axis = -1)#/N
    return JN

In [5]:
def plotRandAdj(JN,axis=-1):
    plt.figure()
    plt.plot(JN[axis,:])
    values, num = np.unique(JN[axis], return_counts=True)
    plt.xlabel('Nodes' , fontsize=15)
    plt.ylabel('Values', fontsize=15)
    plt.figure()
    plt.plot(np.correlate(JN[axis],JN[axis],mode='full'))
    plt.title('Correlation', fontsize=17)
    plt.xlabel('Nodes', fontsize=15)
    plt.figure()
    plt.hist(JN[0],bins=len(values)-1)
    plt.title('Histogram', fontsize=17)
    plt.xlabel('Nodes', fontsize=15)
    plt.ylabel('Frequency', fontsize=15)
    

In [6]:
def consecutive(data, stepsize=1):
    return np.hsplit(data, np.where(np.diff(data) != stepsize)[0]+1)

In [7]:
def order_param(theta, N, k=1):
    """Returns kth order parameter
    """
    Rk = np.sum( np.exp(1j * k * theta.reshape(-1,N)), axis=1 )/N
    return np.abs(Rk), np.angle(Rk)

In [8]:
def plot_LC_with_s(trait):
    plt.plot(trait[::,0],trait[::,1],'.')
    plt.plot(trait[0,0],trait[0,1],'ok')
    plt.xlabel(r'$x$',fontsize=15)
    plt.ylabel(r'$\dot{x}$',fontsize=15)

In [9]:
def make_vanderpl(alpha,omega0_square):
    def vanderpol(X, t):
        x = X[0]
        y = X[1]
        dxdt = y
        dydt = alpha * (1 - x**2)*y-omega0_square*x
        return [dxdt, dydt]
    return vanderpol

In [10]:
def vanDerPolTrajectory(alpha, omega0_square,X0=[2,0],resolution = 2000, t_end = 150):
    t = np.linspace(0,t_end, t_end*resolution+1)
    
    return odeint(make_vanderpl(alpha,omega0_square), X0, t)
    

In [11]:
def TandLC(trajectory, alpha, omega0_square):
    # Neighborhood Radius
    cutoff = np.floor(t_end*resolution*0.2).astype(int)
    T = np.empty(4, dtype=object)
    
    sol_cut = trajectory[-cutoff:]
    peaks, _ = find_peaks(sol_cut[:,0],height=0)
    try:
        ind1= peaks[-2]
        ind2= peaks[-1]
        TrackLC = sol_cut[ind1:ind2+1]
        phaseZ = np.argmax(TrackLC[:,0])
        T[0]= alpha 
        T[1]=omega0_square
        T[2]=((ind2-ind1)/resolution)
        T[3]= np.roll( TrackLC, -phaseZ, axis = 0 )
        return T
        
    except:
        print(alpha)
        print(omega0_square)
        print(peaks)
        T[0]= alpha
        T[1]= omega0_square
        T[2]= np.nan
        T[3]= trajectory#np.nan
        return T

In [12]:
def tableOfLC(alpha, omega0_sqr):
    table = np.empty((len(alpha),len(omega0_sqr),4),dtype=object)
    for idx , alp in enumerate(alpha):
        for idy, ome in enumerate(omega0_sqr):
                    Trajectory = vanDerPolTrajectory(alp,ome,X0,resolution, t_end)
                    table[idx][idy]= TandLC(Trajectory,alp,ome)
    return table

In [13]:
def T_and_LC(alpha, omega0_square):
    X0 = [2, 0]
    resolution = 2000
    t_end = 150
    t = np.linspace(0,t_end, t_end*resolution+1)

    dist_min = 0.008
    cutoff = np.floor(t_end*resolution*0.2).astype(int)
    T = np.empty(4, dtype=object)
    cutoff = np.floor(t_end*resolution*0.5).astype(int)
    
    #Phasenporträt

    sol_in = odeint(make_vanderpl(alpha,omega0_square), X0, t)
    
    sol_cut = sol_in[-cutoff:]
    reference = np.argmin(np.sum((sol_cut[:-1]-sol_cut[1:])**2,1))
    dist_to_last = ((np.sum((sol_cut-sol_cut[reference])**2,1))**(1/2))

    poincare = consecutive(np.where(dist_to_last <= dist_min)[0])
    peaks=np.fromiter((np.mean(xi) for xi in poincare), dtype=np.float, count=len(poincare))
    try:
        ind1= np.floor(peaks[-2]).astype(int)
        ind2= np.floor(peaks[-1]).astype(int)
        TrackLC = sol_cut[ind1:ind2+1]
        phaseZ = np.argmax(TrackLC[:,0])
        T[0]= alpha 
        T[1]=omega0_square
        T[2]=((ind2-ind1)/resolution)
        T[3]= np.roll( TrackLC, -phaseZ, axis = 0 )
        return T
        
    except:
        print(alpha)
        print(omega0_square)
        print(peaks)
        T[0]= alpha
        T[1]= omega0_square
        T[2]= np.nan
        T[3]= sol_in#np.nan
        return T

In [72]:
def phaseInterpol(x):
    peaks, _ = find_peaks(x,height=1,distance=50)#, distance=LC_sample[0,nr,2]/dt*0.6)
#     peaks = np.insert(peaks, 0, 0)
    Ts =np.diff(peaks)
    head =np.linspace(0,2*np.pi,num= Ts[0])[-peaks[0]-1:-1]
    phase = np.concatenate([np.linspace(0,2*np.pi,num=Tn+1)[:-1] for Tn in Ts])
    if peaks[0] != 0:
        phase = np.concatenate((head,phase ))
    while len(phase) < len(x):
        phase = np.concatenate((phase, np.linspace(0,2*np.pi,num=Ts[-1]) ))[:len(x)]
    return phase

In [26]:
alpha1 = 1
X0 = [2, 0]
resolution = 2000
t_end = 150

omega0SquareResolution = 10
omega0SquareStart = 0
omega0SquareEnd = 16
Talpha1 = np.empty(((omega0SquareEnd-omega0SquareStart)*omega0SquareResolution+1,4),dtype=object)

for idx,omega0Square in enumerate(np.linspace(omega0SquareStart,omega0SquareEnd,(omega0SquareEnd
                                                                                 -omega0SquareStart)*omega0SquareResolution+1)) :
    Trajectory = vanDerPolTrajectory(alpha1, omega0Square,X0,resolution, t_end)
    Talpha1[idx]= TandLC(Trajectory,alpha1,omega0Square)




1
0.0
[]
1
0.1
[46159]


In [28]:
np.random.seed(3) 
mean_w = 0.75                            #mean of distribution of natural frequencies of oscillators
std_w = 0.05                           #standard deviation of disribution of natural freqs
N=1000
K = None #will loop over this          #coupling strength
sample = np.random.normal(mean_w, std_w, N)
g0= norm.pdf(0,0,std_w)
Kc=2/(np.pi*g0)
print("Critical Value of Coupling: ",Kc)

Critical Value of Coupling:  0.07978845608028654


In [29]:
NotNanIdx = np.where(~np.isnan(np.asarray(Talpha1[:,2],dtype=np.float64)))

interpolatorOmegaSquareOmegaialpha1 =  interp1d(2*np.pi/Talpha1[:,2][NotNanIdx], (Talpha1[:,1][NotNanIdx]))

TargetOmega0Square1 = np.asarray(interpolatorOmegaSquareOmegaialpha1(sample),dtype=np.float64)

np.any(np.isnan(TargetOmega0Square1))

False

## $\alpha=10$

In [35]:
alpha10 = 10
X0 = [2, 0]
resolution = 2000
t_end = 250

omega0SquareResolution = 10
omega0SquareStart = 0
omega0SquareEnd = 16
Talpha10 = np.empty(((omega0SquareEnd-omega0SquareStart)*omega0SquareResolution+1,4),dtype=object)

for idx,omega0Square in enumerate(np.linspace(omega0SquareStart,omega0SquareEnd,(omega0SquareEnd
                                                                                 -omega0SquareStart)*omega0SquareResolution+1)) :
    Trajectory = vanDerPolTrajectory(alpha10, omega0Square,X0,resolution, t_end)
    Talpha10[idx]= TandLC(Trajectory,alpha10,omega0Square)




10
0.0
[]
10
0.1
[]
10
0.2
[]
10
0.30000000000000004
[65601]
10
0.4
[43492]
10
0.5
[31871]


In [37]:
NotNanIdx = np.where(~np.isnan(np.asarray(Talpha1[:,2],dtype=np.float64)))

interpolatorOmegaSquareOmegaialpha10 =  interp1d(2*np.pi/Talpha10[:,2][NotNanIdx], (Talpha10[:,1][NotNanIdx]))

TargetOmega0Square10 = np.asarray(interpolatorOmegaSquareOmegaialpha10(sample),dtype=np.float64)

np.any(np.isnan(TargetOmega0Square10))

False

## coupled van der Pol

In [90]:
# JN10=RandAdj(N=N,K=10)

In [63]:
# JN100=RandAdj(N=N,K=100)

In [66]:
# JN1000=RandAdj(N=N,K=1000)

In [92]:
# with open('./data/JN_N_1000_K_10.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(JN10, f)

In [92]:
# with open('./data/JN_N_1000_K_100.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(JN100, f)

In [92]:
# with open('./data/JN_N_1000_K_1000.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(JN1000, f)

In [78]:
with open('./data/JN_N_1000_K_10.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    JN10 = pickle.load(f)

In [78]:
with open('./data/JN_N_1000_K_100.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    JN100 = pickle.load(f)

In [78]:
with open('./data/JN_N_1000_K_1000.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    JN100 = pickle.load(f)

# High-Rank

## coupled van der Pol

In [79]:
JN=JN1000

In [80]:
G=JN
L=csgraph.laplacian(G, normed=False)

## $ \dot{x}$ coupled 

In [91]:
def coupled_vanderpol( t,x , K =0,alpha = 30,omega0 = 1):
    n = len(x)//2
    dx0dt = x[n:]
    dx1dt = alpha * (1 - x[:n]**2)*x[n:]-omega0*x[:n]-K/n*np.dot(L,x[n:])
    return np.hstack((dx0dt,dx1dt))

def J_func(t,x, K, alpha, omega0):
#     print('J_func called')
    n = len(x)//2
    return np.vstack(
        (np.hstack((np.zeros((n,n)), np.diag(np.ones(n)))),
        np.hstack((np.diag(-omega0-2*alpha*x[:n]*x[n:]),np.diag(alpha*(1-x[:n]**2))-K/n*L)))
    )

In [92]:
solver = ode(coupled_vanderpol, jac=J_func)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

first = False

t0=0

t_end_o =100
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
t_start = int((t_half)*0.5)
alpha = alpha1

In [83]:
X0=[2]*N+[0]*N

In [93]:
for K in [0.2,5,10]:
    solver.set_f_params(K,alpha,TargetOmega0Square1)
    solver.set_jac_params(K,alpha,TargetOmega0Square1)

    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    with open('./data/pointsnxy_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(pointsnxy, f)
    with open('./data/phasesDyn_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(phasesDyn, f)
    with open('./data/OPs_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPs, f)

## $\alpha=10$

## coupled van der Pol

In [94]:
JN=JN10#RandAdj(N=N,K=N)

In [95]:
# n=len(omega0_sqr)
# G= np.ones((n,n))-np.diag(np.ones(n))
G=JN
L=csgraph.laplacian(G, normed=False)

## $ \dot{x}$ coupled 

In [96]:
def coupled_vanderpol( t,x , K =0,alpha = 30,omega0 = 1):
    n = len(x)//2
    dx0dt = x[n:]
    dx1dt = alpha * (1 - x[:n]**2)*x[n:]-omega0*x[:n]-K/n*np.dot(L,x[n:])
    return np.hstack((dx0dt,dx1dt))

def J_func(t,x, K, alpha, omega0):
#     print('J_func called')
    n = len(x)//2
    return np.vstack(
        (np.hstack((np.zeros((n,n)), np.diag(np.ones(n)))),
        np.hstack((np.diag(-omega0-2*alpha*x[:n]*x[n:]),np.diag(alpha*(1-x[:n]**2))-K/n*L)))
    )

In [97]:
solver = ode(coupled_vanderpol, jac=J_func)
# solver = ode(coupled_vanderpol)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

first = False

t0=0

t_end_o =100

# t_lin = np.linspace(t0, t_end, t_end*resolution+1)
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
t_start = int((t_half)*0.5)
alpha = alpha10

In [98]:
X0=[2]*N+[0]*N

In [101]:
for K in [0.3,2.9,30]:
    solver.set_f_params(K,alpha,TargetOmega0Square10)
    solver.set_jac_params(K,alpha,TargetOmega0Square10)

    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    with open('./data/pointsnxy_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(pointsnxy, f)
    with open('./data/phasesDyn_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(phasesDyn, f)
    with open('./data/OPs_alpha%2.3f_J%2.1f_y_t%3d.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPs, f)


# Low-Rank

## coupled van der Pol

In [79]:
JN=JN10

In [80]:
G=JN
L=csgraph.laplacian(G, normed=False)

## $ \dot{x}$ coupled 

In [91]:
def coupled_vanderpol( t,x , K =0,alpha = 30,omega0 = 1):
    n = len(x)//2
    dx0dt = x[n:]
    dx1dt = alpha * (1 - x[:n]**2)*x[n:]-omega0*x[:n]-K/n*np.dot(L,x[n:])
    return np.hstack((dx0dt,dx1dt))

def J_func(t,x, K, alpha, omega0):
#     print('J_func called')
    n = len(x)//2
    return np.vstack(
        (np.hstack((np.zeros((n,n)), np.diag(np.ones(n)))),
        np.hstack((np.diag(-omega0-2*alpha*x[:n]*x[n:]),np.diag(alpha*(1-x[:n]**2))-K/n*L)))
    )

In [92]:
solver = ode(coupled_vanderpol, jac=J_func)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

first = False

t0=0

t_end_o =100
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
t_start = int((t_half)*0.5)
alpha = alpha1

In [83]:
X0=[2]*N+[0]*N

In [93]:
for K in [0.2,5,20]:
    solver.set_f_params(K,alpha,TargetOmega0Square1)
    solver.set_jac_params(K,alpha,TargetOmega0Square1)

    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    with open('./data/pointsnxy_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(pointsnxy, f)
    with open('./data/phasesDyn_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(phasesDyn, f)
    with open('./data/OPs_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPs, f)

## $\alpha=10$

## coupled van der Pol

In [94]:
JN=JN10#RandAdj(N=N,K=N)

In [95]:
# n=len(omega0_sqr)
# G= np.ones((n,n))-np.diag(np.ones(n))
G=JN
L=csgraph.laplacian(G, normed=False)

## $ \dot{x}$ coupled 

In [96]:
def coupled_vanderpol( t,x , K =0,alpha = 30,omega0 = 1):
    n = len(x)//2
    dx0dt = x[n:]
    dx1dt = alpha * (1 - x[:n]**2)*x[n:]-omega0*x[:n]-K/n*np.dot(L,x[n:])
    return np.hstack((dx0dt,dx1dt))

def J_func(t,x, K, alpha, omega0):
#     print('J_func called')
    n = len(x)//2
    return np.vstack(
        (np.hstack((np.zeros((n,n)), np.diag(np.ones(n)))),
        np.hstack((np.diag(-omega0-2*alpha*x[:n]*x[n:]),np.diag(alpha*(1-x[:n]**2))-K/n*L)))
    )

In [97]:
solver = ode(coupled_vanderpol, jac=J_func)
# solver = ode(coupled_vanderpol)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

first = False

t0=0

t_end_o =100

# t_lin = np.linspace(t0, t_end, t_end*resolution+1)
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
t_start = int((t_half)*0.5)
alpha = alpha10

In [98]:
X0=[2]*N+[0]*N

In [101]:
for K in [0.3,10,20]:
    solver.set_f_params(K,alpha,TargetOmega0Square10)
    solver.set_jac_params(K,alpha,TargetOmega0Square10)

    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    with open('./data/pointsnxy_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(pointsnxy, f)
    with open('./data/phasesDyn_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(phasesDyn, f)
    with open('./data/OPs_alpha%2.3f_J%2.1f_y_t%3d_K10.pkl'%(alpha,K,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPs, f)
